In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [3]:
# load the dataset
df = pd.read_csv('drake_lyric_data.csv')
df = df.dropna()

# drop non-numerical columns except for 'lyrics'
audio_features = df.drop(columns=['name', 'album', 'year', 'popularity', 'lyrics'])
lyrics = df['lyrics']

In [4]:
# normalize audio features
scaler = MinMaxScaler()
audio_features_scaled = scaler.fit_transform(audio_features)

# tokenize the lyrics
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lyrics)
total_words = len(tokenizer.word_index) + 1

# convert lyrics to sequences
input_sequences = []
for line in lyrics:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:
# split data into predictors and labels
X_text, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# repeat audio features to match the number of sequences
repeated_audio_features = np.repeat(audio_features_scaled, len(X_text) // len(audio_features_scaled) + 1, axis=0)[:len(X_text)]
X_audio = np.repeat(repeated_audio_features[:, np.newaxis, :], X_text.shape[1], axis=1)

# combine text and audio features
X = np.concatenate([X_text.reshape(X_text.shape[0], X_text.shape[1], 1), X_audio], axis=-1)

In [ ]:
# create
model = Sequential()
model.add(LSTM(100, input_shape=(max_sequence_len-1, X.shape[2])))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train
history = model.fit(X, y, epochs=100, verbose=1)